In [ ]:
import numpy as np
import random
import statistics
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
class conflict_organism:
    def __init__(self, strength):
        self.strength = strength

    def __str__(self):
        return f"{self.strength}"

    def breed(self):
        global conflict_organism_pop
        strength1 = np.round(self.strength + np.random.randn()*1.5)
        if np.random.rand()>len(conflict_organism_pop)/1000:
            conflict_organism_pop.append(conflict_organism(strength1))

In [ ]:
class organism:
    def __init__(self, strength, period, gender, family):
        self.strength = strength
        self.period = period
        self.gender = gender
        self.family=family

    def __str__(self):
            return f"{self.gender},{self.strength}({self.period})"

In [ ]:
#ranking
def reproduce():
    global m_r
    global population
    global conflict_organism_pop
    global child_population


    child_population=[]
 
    herm_pop=[p for p in population if p.gender == 'hermaphrodite'] 
    other_pop=[p for p in population if p.gender != 'hermaphrodite']
    female_pop=[p for p in population if p.gender == 'female']
    other_pop_len = len(female_pop)
    population=[]

    while len(child_population)<other_pop_len:
        
        random_org1=np.random.choice(range(len(other_pop)))
        single_parent=other_pop[random_org1]
        
        gender1=single_parent.gender

        if np.random.rand()<m_r:
            if np.random.rand()<0.5:
                gender1='hermaphrodite'
            else:
                if gender1 == 'male':
                    gender1='female'
                else:
                    gender1='male'

    
        strength1 = np.round(single_parent.strength + np.random.randn(), decimals=2)

        
        fam=single_parent.family
    
        child1 = organism(strength1, 1, gender1, fam)
        population.append(single_parent)
        other_pop.remove(other_pop[random_org1])
        child_population.append(child1)
        

    for single_parent in herm_pop:
    
        gender1='hermaphrodite'

        if np.random.rand()<m_r:
            if np.random.rand()<0.5:
                gender1='male'
            else:
                gender1='female'

    
        strength1 = np.round(single_parent.strength + np.random.randn(), decimals=2)

        
        fam=single_parent.family
    
        child1 = organism(strength1, 1, gender1, fam)
        child_population.append(child1)

    for i in other_pop:
        population.append(i)
            
    for i in herm_pop:
        population.append(i)

In [ ]:
def hunt(org):
    global conflict_organism_pop
    global population

    if len(conflict_organism_pop)>0:
        random_conflict_organism=np.random.choice(range(len(conflict_organism_pop)))
        if org.strength >= conflict_organism_pop[random_conflict_organism].strength+np.random.randn()/50:
            conflict_organism_pop.remove(conflict_organism_pop[random_conflict_organism])
            return 1
        else:
            return 0
    else:
        return 0

In [ ]:
def single_parent_hunt(org):
    global conflict_organism_pop
    global population
    global child_population

    if len(conflict_organism_pop)>0:
    
        child =[p for p in child_population if p.family == org.family]
    
        if len(child)>0:
            random_conflict_organism=np.random.choice(range(len(conflict_organism_pop)))
    
            if hunt(org) == 1:
                population.append(child[0])
                    
                if hunt(org) == 1:
                    return 1
                else:
                    return 0
            else:
                return 0
        else:
            if hunt(org) == 1:
                return 1
            else:
                return 0
    else:
        return 0

In [ ]:
def hunt_and_feed():
    global conflict_organism_pop
    global population
    global child_population


    hunting_population=population.copy()
    random.shuffle(hunting_population)
    population=[]

    for adult in hunting_population:
        if single_parent_hunt(adult) == 1:
            population.append(adult)
    
    for org in population:
        org.period+=1
    population=[p for p in population if p.period <= 4]

    random.shuffle(conflict_organism_pop)
    for conflict_org in conflict_organism_pop:
        conflict_org.breed()

    k=0
    for org in population:
        org.family=k
        k+=1

In [ ]:
#total number of times you run the simulation
number_of_total_iter = 40
#number of generations per regression
number_of_iter=2000
#mutation rate
m_r=0.05

fig, axes = plt.subplots(int(number_of_total_iter/4), 4, figsize=(24, 5*number_of_total_iter/4))
axes = axes.flatten()

invalid_iters=0
max_share=0
shares=[]
x=0
while x<number_of_total_iter:

    valid=1
    share_plot=[]
    population_plot=[]
    n = 100
    n_conflict_organism=300
    population=[]
    conflict_organism_pop=[]
    share0=0
    
    for i in range(n):
        strength=np.round(10+np.random.randn()*5)
        age=np.round(1+np.random.exponential(2))
    
        if age<2:
            age=2
        if age>4:
            age=4
        if np.random.rand()<m_r:
            share0+=100/n
            if np.random.rand()<0.5:
                gender='male'
            else:
                gender='female'
        else:
            gender='hermaphrodite'
            
        new_organism=organism(strength, age, gender, i)
        population.append(new_organism)
    
    for i in range(n_conflict_organism):
        strength=np.round(12+np.random.randn()*5)
        if strength<1:
            strength=1
        new_conflict_organism=conflict_organism(strength)
        conflict_organism_pop.append(new_conflict_organism)
    
    share_plot.append(share0)
    number=0
    long_term_trend=[]
    long_long_term_trend=[]
    
    
    while number<number_of_iter:
        reproduce()
        hunt_and_feed()
        if len(population)<=0:
            valid=0
            break
        if len(conflict_organism_pop)<=0:
            valid=0
            break
    
        str_conflict_organism=0
        for i in conflict_organism_pop:
            str_conflict_organism+=i.strength
    
        males=[p for p in population if p.gender == 'male']
        females=[p for p in population if p.gender == 'female']
        hermaphrodites=[p for p in population if p.gender == 'hermaphrodite']
        st_males=0
    
        for i in males:
            st_males+=i.strength
        st_females=0
        for i in females:
            st_females+=i.strength
        st_hermaphrodites=0
        for i in hermaphrodites:
            st_hermaphrodites+=i.strength
        
        male_share=np.round(len(males)/len(population), decimals=3)
        female_share=np.round(len(females)/len(population), decimals=3)
        share=np.round(100*(len(males)+len(females))/len(population), decimals=1)
        long_term_trend.append(share)
        long_long_term_trend.append(share)
        share_plot.append(share)
        population_plot.append(len(population)) 
    
        number+=1

    if valid ==0:
        invalid_iters+=1
        print('invalid iterations', invalid_iters)
    else:
        final_share=np.round(statistics.mean(share_plot[-1001:-1]), decimals=3)
        axes[x].plot(share_plot[-1001:-1])
        axes[x].set_title(str(x+1) + '. Share of other hermaphrodite population (in %)')
        axes[x].set_xlabel('Average Share =' + str(final_share), fontsize=8)
        shares.append(final_share)
        if final_share>max_share:
            max_share=final_share
        print(x+1)
        x+=1

print('number of invalid iterations:', invalid_iters)
print('average share:', np.round(statistics.mean(shares),decimals=3))
print('maximum share:', max_share)

#plt.savefig('hermaphrodite_tests_ordered.png')

plt.show()

In [ ]:
from scipy import stats

mean = np.mean(shares)
std = np.std(shares, ddof=1)
sem = std / np.sqrt(number_of_total_iter)
alpha = 0.01
df = number_of_total_iter - 1
t_crit = stats.t.ppf(1 - alpha/2, df)
margin_of_error = t_crit * sem
ci_lower = mean - margin_of_error
ci_upper = mean + margin_of_error

print('[', ci_lower, ', ', ci_upper, ']')